# Geocoding raw address data for the Who Owns Hoboken? app

This script utilizes the Nominatim API and geopy to append latitude and longitude to the raw data compiled for the Who Owns Hoboken? app.

In [1]:
## Import libraries
import pandas as pd
import numpy as np
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from pandas_geojson import to_geojson
from pandas_geojson import write_geojson
pd.options.mode.chained_assignment = None

In [2]:
## Read data
data = pd.read_csv('./raw_data.csv')
data.head()

,Block,Lot,Qual,Class,Property Location,Building name,LLC/Owner,LLC or private individual,Total Units,Company,...,Building Class,Prior Block,Prior Lot,Prior Qual,Updated,Additional Lots,Rent Control,Building Desc,units2,Sale Date
0,75.0,1.0,C004D,2,601 MONROE ST,NaN,"OCONNOR, JAMES P JR",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,2BED,1.0,0000-00-00
1,219.0,8.0,C0002,2,815 WASHINGTON ST,NaN,"OCONNOR, JAMES & BERNADETTE",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,2BR,1.0,8/8/11
2,207.0,25.0,NaN,2,808 WASHINGTON ST,NaN,808 WASHINGTON ST LLC,LLC,4.0,NaN,...,49.0,NaN,NaN,NaN,12/27/19,NaN,YES,4B-4U-H-BA,4.0,8/16/13
3,180.0,6.0,NaN,2,511 PARK AVE,NaN,"RAD, MOHAMMAD TRUSTEE OF JOODI",Trust,2.0,Rad,...,49.0,NaN,NaN,NaN,12/27/19,NaN,YES,3B-2U-FX-H,2.0,5/12/14
4,88.0,1.0,NaN,4C,800 MADISON/801 MONROE ST,Avalon,DSF IV HOBOKEN OWNER LLC % AVALON B,LLC,220.0,AvalonBay,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,6B-220U-4C-G,220.0,NaN


In [3]:
data.columns = data.columns.map(lambda x : x.replace('.', '').replace(' ', '_').replace('/','_').replace("'", '')) 
data.columns = data.columns.str.lower()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   block                      2773 non-null   float64
 1   lot                        2773 non-null   float64
 2   qual                       1067 non-null   object 
 3   class                      2773 non-null   object 
 4   property_location          2773 non-null   object 
 5   building_name              60 non-null     object 
 6   llc_owner                  2773 non-null   object 
 7   llc_or_private_individual  2773 non-null   object 
 8   total_units                2772 non-null   float64
 9   company                    1012 non-null   object 
 10  owners_mailing_address     2773 non-null   object 
 11  city_state_zip             2773 non-null   object 
 12  sq_ft                      2411 non-null   float64
 13  yr_built                   2052 non-null   float

In [5]:
## Make address all uppercase
data['property_location'] = data['property_location'].str.upper()

In [6]:
## Clean up street names so geocoder can recognize them 
data['property_location'] = data['property_location'].str.replace('FIRST', '1ST')
data['property_location'] = data['property_location'].str.replace('SECOND', '2ND')
data['property_location'] = data['property_location'].str.replace('THIRD', '3RD')
data['property_location'] = data['property_location'].str.replace('FOURTH ', '4TH ')
data['property_location'] = data['property_location'].str.replace('FIFTH', '5TH')
data['property_location'] = data['property_location'].str.replace('SIXTH', '6TH')
data['property_location'] = data['property_location'].str.replace('SEVENTH', '7TH')
data['property_location'] = data['property_location'].str.replace('EIGHTH', '8TH')
data['property_location'] = data['property_location'].str.replace('EIGHT ', '8TH ')
data['property_location'] = data['property_location'].str.replace('NINTH', '9TH')
data['property_location'] = data['property_location'].str.replace('TENTH', '10TH')
data['property_location'] = data['property_location'].str.replace('ELEVENTH', '11TH')
data['property_location'] = data['property_location'].str.replace('TWELFTH', '12TH')
data['property_location'] = data['property_location'].str.replace('THIRTEENTH', '13TH')
data['property_location'] = data['property_location'].str.replace('FOURTEENTH', '14TH')
data['property_location'] = data['property_location'].str.replace('FIFTEENTH', '15TH')
data['property_location'] = data['property_location'].str.replace('WASH ', 'WASHINGTON')
data['property_location'] = data['property_location'].str.replace('WASHINGTONST ', 'WASHINGTON ST')
data['property_location'] = data['property_location'].str.replace('0BSERVER ', 'OBSERVER')
data['property_location'] = data['property_location'].str.replace('OBSERVERHWY ', 'OBSERVER HWY')
data['property_location'] = data['property_location'].str.replace('OBSERVERHWY', 'OBSERVER HWY')
data['property_location'] = data['property_location'].str.replace('THS T ', 'TH ST')
data['property_location'] = data['property_location'].str.replace('ST REAR', 'ST')
data['property_location'] = data['property_location'].str.replace('BLOOM ', 'BLOOMFIELD')
data['property_location'] = data['property_location'].str.replace('THS T', 'TH ST')
data['property_location'] = data['property_location'].str.replace(' AT', ' ST')
data['property_location'] = data['property_location'].str.replace('WASHINGTONST', 'WASHINGTON ST')
data['property_location'] = data['property_location'].str.replace('ST230', 'ST')

In [7]:
## Clean up addresses with notes in them
sep = '('
data['property_location'] = data['property_location'].str.split(sep).str[0]

In [8]:
## Clean up addresses with two locations combined
sep = '/'
data['clean_address'] = data['property_location'].str.split(sep).str[0]

In [9]:
## Clean up addresses named with intersection
sep = '&'
data['clean_address'] = data['clean_address'].str.split(sep).str[0]

In [10]:
## Create column to be used for address lookup in Nominatim API
data['search_address'] = data['clean_address'] + ', Hoboken, NJ, 07030'
data.head()

,block,lot,qual,class,property_location,building_name,llc_owner,llc_or_private_individual,total_units,company,...,prior_lot,prior_qual,updated,additional_lots,rent_control,building_desc,units2,sale_date,clean_address,search_address
0,75.0,1.0,C004D,2,601 MONROE ST,NaN,"OCONNOR, JAMES P JR",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,YES,2BED,1.0,0000-00-00,601 MONROE ST,"601 MONROE ST, Hoboken, NJ, 07030"
1,219.0,8.0,C0002,2,815 WASHINGTON ST,NaN,"OCONNOR, JAMES & BERNADETTE",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,YES,2BR,1.0,8/8/11,815 WASHINGTON ST,"815 WASHINGTON ST, Hoboken, NJ, 07030"
2,207.0,25.0,NaN,2,808 WASHINGTON ST,NaN,808 WASHINGTON ST LLC,LLC,4.0,NaN,...,NaN,NaN,12/27/19,NaN,YES,4B-4U-H-BA,4.0,8/16/13,808 WASHINGTON ST,"808 WASHINGTON ST, Hoboken, NJ, 07030"
3,180.0,6.0,NaN,2,511 PARK AVE,NaN,"RAD, MOHAMMAD TRUSTEE OF JOODI",Trust,2.0,Rad,...,NaN,NaN,12/27/19,NaN,YES,3B-2U-FX-H,2.0,5/12/14,511 PARK AVE,"511 PARK AVE, Hoboken, NJ, 07030"
4,88.0,1.0,NaN,4C,800 MADISON/801 MONROE ST,Avalon,DSF IV HOBOKEN OWNER LLC % AVALON B,LLC,220.0,AvalonBay,...,NaN,NaN,NaN,NaN,YES,6B-220U-4C-G,220.0,NaN,800 MADISON,"800 MADISON, Hoboken, NJ, 07030"


In [11]:
## Define Nominatim API instance
locator = Nominatim(user_agent='who_owns_hoboken')

In [12]:
## Call API for each row of dataset
## Store output from API call in 'location column'
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
data['location'] = data['search_address'].apply(geocode)

In [13]:
## Extract lat/long information from API output
data['point'] = data['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [14]:
## Turn lat/long information into new columns in the dataset
data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [15]:
## save results of geocoding
data.to_csv('./data/hoboken_landlords.csv')

In [16]:
## run this if you want to start from geocoded data
#with open('./data/hoboken_landlords.csv') as x:
    #ncols = len(x.readline().split(','))

#data = pd.read_csv('./data/hoboken_landlords.csv', usecols=range(1,ncols))

In [17]:
data.groupby('latitude').size().sort_values(ascending = False)

latitude
40.750362    59
40.749529    40
40.752481    34
40.746050    31
40.752710    25
             ..
40.742083     1
40.742080     1
40.742043     1
40.742034     1
40.754934     1
Length: 2064, dtype: int64

In [18]:
## Manually geocode these addresses
data[data['latitude'].isnull()]['property_location'].unique()

array(['2 CONSTITUTION COURT', '1 INDEPENDENCE COURT',
       '1200-1222 HUDSON ST', '1201-1333 HUDSON ST',
       '1201-21R WASHINGTON ST', '224-232 RIVER 235 HUDSON',
       '401-03 WASHINGTON ST', '300-08 RIVER ST', '11 ELYSIAN PLACE'],
      dtype=object)

In [19]:
# 2 Constitution Court
data.loc[data['property_location'] == '2 CONSTITUTION COURT', 'latitude'] = 40.75235433173578
data.loc[data['property_location'] == '2 CONSTITUTION COURT', 'longitude'] = -74.02416413263948
# 1 Independence Court
data.loc[data['property_location'] == '1 INDEPENDENCE COURT', 'latitude'] = 40.7517113622344
data.loc[data['property_location'] == '1 INDEPENDENCE COURT', 'longitude'] = -74.0243229423285
# '1200-1222 HUDSON ST'
data.loc[data['property_location'] == '1200-1222 HUDSON ST', 'latitude'] = 40.751547793135835
data.loc[data['property_location'] == '1200-1222 HUDSON ST', 'longitude'] = -74.02566382893585
# '1201-1333 HUDSON ST'
data.loc[data['property_location'] == '1201-1333 HUDSON ST', 'latitude'] = 40.751327435663356
data.loc[data['property_location'] == '1201-1333 HUDSON ST', 'longitude'] = -74.02529360594718
# '1201-21R WASHINGTON ST'
data.loc[data['property_location'] == '1201-21R WASHINGTON ST', 'latitude'] = 40.75167892656019
data.loc[data['property_location'] == '1201-21R WASHINGTON ST', 'longitude'] = -74.02620949828147
# '224-232 RIVER 235 HUDSON'
data.loc[data['property_location'] == '224-232 RIVER 235 HUDSON', 'latitude'] = 40.7394286430441
data.loc[data['property_location'] == '224-232 RIVER 235 HUDSON', 'longitude'] = -74.0286408719967
# '401-03 WASHINGTON ST'
data.loc[data['property_location'] == '401-03 WASHINGTON ST', 'latitude'] = 40.74134653174973
data.loc[data['property_location'] == '401-03 WASHINGTON ST', 'longitude'] = -74.02950291302936
# '300-08 RIVER ST'
data.loc[data['property_location'] == '300-08 RIVER ST', 'latitude'] = 40.74002433251577
data.loc[data['property_location'] == '300-08 RIVER ST', 'longitude'] = -74.02844601055078
# '11 ELYSIAN PLACE'
data.loc[data['property_location'] == '11 ELYSIAN PLACE', 'latitude'] = 40.747902005692055
data.loc[data['property_location'] == '11 ELYSIAN PLACE', 'longitude'] = -74.0248531253788

In [20]:
## manually update rent control status based on user feedback
data.loc[data['property_location'] == '800 MADISON/801 MONROE ST', 'rent_control'] = 'YES'

In [21]:
## add column for what year the building falls under rent control
data.loc[data['yr_built'] == 0, 'yr_built'] = np.nan
data['yr_built'] = data['yr_built'].astype('Int64')
data['rent_control_year'] = data['yr_built'] + 30

In [22]:
## convert units columns to integers
data['total_units'] = data['total_units'].astype('Int64')
data['units2'] = data['units2'].astype('Int64')

In [24]:
## drop unneccesary columns
drop_cols = ['class', 'building_class', 'prior_block', 'prior_lot', 'prior_qual', 'updated',
       'additional_lots', 'search_address', 'location', 'point', 'altitude']
data = data.drop(drop_cols, axis = 1)

In [25]:
## Change rent control column to binary
data.loc[data['rent_control'] != 'YES', 'rent_control'] = 'Unconfirmed'

In [26]:
## Make Company = Owner when Company is blank
data.loc[data['company'].isna(), 'company'] = data['llc_owner']

In [27]:
## Replace NAs with 'None'- required for geojson conversion
data = data.fillna(np.nan).replace([np.nan], [None])

In [28]:
## 28% of units are rent controlled
## this may be a slight undercount as I think the data is missing some condo units from RC buildings
data.groupby('rent_control')['units2'].sum() 

rent_control
Unconfirmed    11766
YES             4563
Name: units2, dtype: object

In [29]:
## 56% of buildings are rent controlled
data.groupby('rent_control').size()

rent_control
Unconfirmed    1207
YES            1566
dtype: int64

In [30]:
## split data into condos and non-condos
condos = data[data['qual'].str.startswith('C') == True]
apts = data[data['qual'].str.startswith('C') != True]

In [31]:
## define properties for building geojsons
properties = list(data.columns)
properties.remove('latitude')
properties.remove('longitude')
properties

['block',
 'lot',
 'qual',
 'property_location',
 'building_name',
 'llc_owner',
 'llc_or_private_individual',
 'total_units',
 'company',
 'owners_mailing_address',
 'city_state_zip',
 'sq_ft',
 'yr_built',
 'rent_control',
 'building_desc',
 'units2',
 'sale_date',
 'clean_address',
 'rent_control_year']

In [45]:
## define properties for landlord geojson
landlord_cols = ['property_location', 'building_name', 'llc_owner', 'llc_or_private_individual','company', 'yr_built',
                'rent_control', 'units2']

In [46]:
## convert data to geojson
condos_geojson = to_geojson(df=condos, lat='latitude', lon='longitude',
                 properties=properties)

apts_geojson = to_geojson(df=apts, lat='latitude', lon='longitude',
                 properties=properties)
landlords_geojson = to_geojson(df=data, lat='latitude', lon='longitude',
                 properties=landlord_cols)

In [47]:
## save geojson files in data folder
write_geojson(condos_geojson, filename='./data/condos.geojson', indent=4)
write_geojson(apts_geojson, filename='./data/apts.geojson', indent=4)
write_geojson(landlord_geojson, filename='./data/landlords.geojson', indent=4)

In [54]:
x = data.T.to_dict(orient = 'list')
x

{0: [75.0,
  1.0,
  'C004D',
  '601 MONROE ST',
  None,
  'OCONNOR, JAMES P JR',
  'Individual',
  2,
  "James O'Connor",
  '355 NEWCASTLE DRIVE',
  'ALPHARETTA, GA  30004',
  646.0,
  1900,
  'YES',
  '2BED',
  1,
  '0000-00-00',
  '601 MONROE ST',
  40.7451517,
  -74.0380829,
  1930],
 1: [219.0,
  8.0,
  'C0002',
  '815 WASHINGTON ST',
  None,
  'OCONNOR, JAMES & BERNADETTE',
  'Individual',
  2,
  "James O'Connor",
  '355 NEWCASTLE DRIVE',
  'ALPHARETTA, GA  30009',
  1061.0,
  1870,
  'YES',
  '2BR',
  1,
  '8/8/11',
  '815 WASHINGTON ST',
  40.7465502125,
  -74.0279596125,
  1900],
 2: [207.0,
  25.0,
  None,
  '808 WASHINGTON ST',
  None,
  '808 WASHINGTON ST LLC',
  'LLC',
  4,
  '808 WASHINGTON ST LLC',
  'PO BOX 476',
  'ALPINE, NJ  07620',
  5375.0,
  1920,
  'YES',
  '4B-4U-H-BA',
  4,
  '8/16/13',
  '808 WASHINGTON ST',
  40.746369449999996,
  -74.028296225,
  1950],
 3: [180.0,
  6.0,
  None,
  '511 PARK AVE',
  None,
  'RAD, MOHAMMAD TRUSTEE OF JOODI',
  'Trust',
  2,
  